In [1]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import requests
from config import weather_api_key

In [2]:
# Random latitude and longitude combos
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
#List for Lats/Lngs
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
#list for holding the cities
cities = []
# Nearest city for each latitude and longitude combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
len(cities)

753

In [6]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=b3a91bb66e4b29f47822c3bd361becd5


In [10]:
city_data = []

record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": current_description})

# If an error is experienced, skip the city.
    except Exception as e:
        print(e)
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | qaanaaq
Processing Record 2 of Set 1 | tasiilaq
Processing Record 3 of Set 1 | cam ranh
Processing Record 4 of Set 1 | longyearbyen
Processing Record 5 of Set 1 | punta arenas
Processing Record 6 of Set 1 | new norfolk
Processing Record 7 of Set 1 | jamestown
Processing Record 8 of Set 1 | ribeira grande
Processing Record 9 of Set 1 | fort smith
Processing Record 10 of Set 1 | saleaula
'coord'
Processing Record 11 of Set 1 | lavrentiya
Processing Record 12 of Set 1 | paradwip
'coord'
Processing Record 13 of Set 1 | puerto ayora
Processing Record 14 of Set 1 | kodiak
Processing Record 15 of Set 1 | vestmannaeyjar
Processing Record 16 of Set 1 | port alfred
Processing Record 17 of Set 1 | edson
Processing Record 18 of Set 1 | margate
Processing Record 19 of Set 1 | ahmadpur
Processing Record 20 of Set 1 | jakarta
Processing Record 21 of Set 1 | taolanaro
'coord'
Processing Record 22 of Set 1 | okhotsk
Processing Record 23 of Set 1 | srednekolymsk
Processing

In [11]:
len(city_data)

694

In [14]:
#Convert array of dictionaries to Pandas DF
city_data_df = pd.DataFrame(city_data)
city_data_df.head(5)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Qaanaaq,GL,77.4840,-69.3632,22.89,71,100,7.83,overcast clouds
1,Tasiilaq,GL,65.6145,-37.6368,41.16,73,100,2.15,overcast clouds
2,Cam Ranh,VN,11.9214,109.1591,76.96,94,40,3.44,light rain
3,Longyearbyen,SJ,78.2186,15.6401,42.64,81,75,8.05,broken clouds
4,Punta Arenas,CL,-53.1500,-70.9167,44.71,49,75,21.85,broken clouds


In [15]:
#Output file
output_data_file = "WeatherPy_Database.csv"
#Export city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")